In [1]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import psycopg2
from getpass import getpass

In [2]:
password2 = getpass('database password')

database password········


In [3]:
import pandas.io.sql as psql
connection = psycopg2.connect(
# make sure you use your local database name, user and host (should be localhost by default)
            host="localhost",
            database="Canadian_Election_Study",
            user="postgres",
            password=password2)
df = psql.read_sql('SELECT * FROM survey_data', connection)

In [4]:
df = df[df.cps19_votechoice != "Don't know/ Prefer not to answer"]
df = df[df.cps19_votechoice != "Another party (please specify)"]
df['cps19_votechoice'].replace({'Bloc Qu<e9>b<e9>cois':'Bloc Quebecois'}, inplace=True)
df.head(5)

,cps19_responseid,cps19_citizenship,cps19_yob,cps19_gender,cps19_province,cps19_education,cps19_demsat,cps19_interest_gen_1,cps19_interest_elxn_1,cps19_v_likely,...,cps19_language_vietnamese,cps19_language_no_answer,cps19_language_aborginal,cps_19_language_other,cps19_employment,cps19_union,cps19_children,cps19_income_number,cps19_marital,cps19_household
1,R_3j7fAVYfVCewi3H,Canadian citizen,2000,A woman,Ontario,Some university,Fairly satisfied,10.0,10.0,Certain to vote,...,No,No,No,No,Student and working for pay,No,No,30000.0,Never Married,5
2,R_brdMqsPTvQ5t1tL,Canadian citizen,2000,A woman,Ontario,Completed secondary/ high school,Fairly satisfied,8.0,8.0,Certain to vote,...,No,No,No,No,Working for pay part-time,No,No,13000.0,Never Married,2
3,R_Wumhl7QEMURFqZH,Canadian citizen,2000,A man,Ontario,Completed secondary/ high school,Fairly satisfied,9.0,8.0,Certain to vote,...,No,No,No,No,Student and working for pay,No,No,55000.0,Never Married,3
4,R_3EH051N9vLmOOHM,Canadian citizen,2000,A woman,Ontario,Completed secondary/ high school,Fairly satisfied,10.0,10.0,Certain to vote,...,No,No,No,No,Student,No,No,190000.0,Never Married,4
5,R_1FynCZH7i2j3zEY,Canadian citizen,1999,A man,Ontario,Completed secondary/ high school,Fairly satisfied,10.0,10.0,Certain to vote,...,No,No,No,Hebrew,Working for pay part-time,No,No,10000.0,Never Married,2


In [ ]:
df.columns

In [5]:
columns = list(df.columns)
columns
target = ['cps19_votechoice']

In [6]:
df.dtypes

cps19_responseid        object
cps19_citizenship       object
cps19_yob                int64
cps19_gender            object
cps19_province          object
                        ...   
cps19_union             object
cps19_children          object
cps19_income_number    float64
cps19_marital           object
cps19_household          int64
Length: 86, dtype: object

In [6]:
clean_df = df.drop(['cps19_responseid', 'cps19_votechoice', 'cps19_ethnicity_no_answer','cps19_ethnicity_other_1', 'cps19_ethnicity_other_2', 'cps19_language_no_answer', 'cps_19_language_other'], axis=1)

In [7]:
clean_df.drop(['cps19_language_arabic', 'cps19_language_chinese',
       'cps19_language_filipino', 'cps19_language_german',
       'cps19_language_indian', 'cps19_language_italian',
       'cps19_language_korean', 'cps19_language_pakistani',
       'cps19_language_persian', 'cps19_language_russian',
       'cps19_language_spanish', 'cps19_language_tamil',
       'cps19_language_vietnamese','cps19_language_aborginal'], axis=1, inplace=True)

In [8]:
clean_df.drop('cps19_citizenship', axis=1, inplace=True)

In [9]:
clean_df.drop(['cps19_ethnicity_first_nat',
       'cps19_ethnicity_british', 'cps19_ethnicity_chinese',
       'cps19_ethnicity_dutch', 'cps19_ethnicity_english',
       'cps19_ethnicity_french', 'cps19_ethnicity_french_can',
       'cps19_ethnicity_german', 'cps19_ethnicity_hispanic',
       'cps19_ethnicity_indian', 'cps19_ethnicity_inuk_inuit',
       'cps19_ethnicity_irish', 'cps19_ethnicity_italian',
       'cps19_ethnicity_metis', 'cps19_ethnicity_polish',
       'cps19_ethnicity_quebecois', 'cps19_ethnicity_scottish',
       'cps19_ethnicity_ukranian'], axis=1, inplace=True)

In [10]:
dummies = pd.get_dummies(clean_df, columns=['cps19_gender', 'cps19_province', 'cps19_education', 'cps19_demsat', 
                                            'cps19_v_likely', 'cps19_fed_gov_sat','cps19_spend_educ', 'cps19_spend_env', 
                                            'cps19_spend_just_law',
       'cps19_spend_defence', 'cps19_spend_imm_min', 'cps19_econ_retro',
       'cps19_econ_fed', 'cps19_ownfinanc_fed', 'cps19_imm', 'cps19_refugees',
       'cps19_govt_confusing', 'cps19_govt_say', 'cps19_pol_eth',
       'cps19_lib_promises', 'cps19_snclav', 'cps19_volunteer',
       'cps19_duty_choice', 'cps19_own_fin_retro', 'cps19_prov_gov_sat',
       'cps19_turnout_2015','cps19_religion', 'cps19_bornin_canada',  'cps19_sexuality',
       'cps19_language_english', 'cps19_language_french',
     'cps19_employment',
       'cps19_union', 'cps19_children', 'cps19_marital'])

In [32]:
dummies.head()

,cps19_yob,cps19_interest_gen_1,cps19_interest_elxn_1,cps19_lr_scale,cps19_groups_therm_1,cps19_groups_therm_2,cps19_groups_therm_3,cps19_groups_therm_4,cps19_groups_therm_5,cps19_income_number,...,cps19_children_Don't know/ Prefer not to answer,cps19_children_No,cps19_children_Yes,cps19_marital_Divorced,cps19_marital_Don't know/ Prefer not to answer,cps19_marital_Living with a partner,cps19_marital_Married,cps19_marital_Never Married,cps19_marital_Separated,cps19_marital_Widowed
1,2000,10.0,10.0,0,100,100,100,100,70,30000.0,...,0,1,0,0,0,0,0,1,0,0
2,2000,8.0,8.0,6,97,73,62,95,80,13000.0,...,0,1,0,0,0,0,0,1,0,0
3,2000,9.0,8.0,6,48,39,15,15,29,55000.0,...,0,1,0,0,0,0,0,1,0,0
4,2000,10.0,10.0,10,100,100,0,81,77,190000.0,...,0,1,0,0,0,0,0,1,0,0
5,1999,10.0,10.0,10,50,50,50,10,40,10000.0,...,0,1,0,0,0,0,0,1,0,0


In [11]:
list(dummies.columns)

['cps19_yob',
 'cps19_interest_gen_1',
 'cps19_interest_elxn_1',
 'cps19_lr_scale',
 'cps19_groups_therm_1',
 'cps19_groups_therm_2',
 'cps19_groups_therm_3',
 'cps19_groups_therm_4',
 'cps19_groups_therm_5',
 'cps19_income_number',
 'cps19_household',
 'cps19_gender_A man',
 'cps19_gender_A woman',
 'cps19_gender_Other (e.g. Trans, non-binary, two-spirit, gender-queer)',
 'cps19_province_Alberta',
 'cps19_province_British Columbia',
 'cps19_province_Manitoba',
 'cps19_province_New Brunswick',
 'cps19_province_Newfoundland and Labrador',
 'cps19_province_Northwest Territories',
 'cps19_province_Nova Scotia',
 'cps19_province_Nunavut',
 'cps19_province_Ontario',
 'cps19_province_Prince Edward Island',
 'cps19_province_Quebec',
 'cps19_province_Saskatchewan',
 'cps19_province_Yukon',
 "cps19_education_Bachelor's degree",
 'cps19_education_Completed elementary school',
 'cps19_education_Completed secondary/ high school',
 'cps19_education_Completed technical, community college, CEGEP, Col

In [11]:
X = dummies

In [ ]:
df.describe()

In [12]:
y = df['cps19_votechoice']

In [ ]:
X.describe()

In [ ]:
y.value_counts()

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=28)
Counter(y_train)

Counter({'Bloc Quebecois': 467,
         'Liberal Party': 3168,
         'ndp': 1371,
         'Conservative Party': 2823,
         'Green Party': 814,
         "People's Party": 216})

In [23]:
Counter(y_test)

Counter({'Liberal Party': 1033,
         'Bloc Quebecois': 163,
         'ndp': 480,
         'Conservative Party': 944,
         'Green Party': 261,
         "People's Party": 72})

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=28)
rf_model = rf_model.fit(X_train, y_train)

In [37]:
from sklearn.metrics import balanced_accuracy_score
y_pred = rf_model.predict(X_test)
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score

0.4335604732456693

In [17]:
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Liberal Party", "Conservative Party", "ndp", "Green Party", "Bloc Qu<e9>b<e9>cois", "People's Party"], columns=["Liberal Party", "Conservative Party", "ndp", "Green Party", "Bloc Qu<e9>b<e9>cois", "People's Party"])
cm_df

,Liberal Party,Conservative Party,ndp,Green Party,Bloc Qu<e9>b<e9>cois,People's Party
Liberal Party,77,34,0,46,0,6
Conservative Party,20,834,1,82,0,7
ndp,16,78,7,109,0,51
Green Party,5,50,2,933,0,43
Bloc Qu<e9>b<e9>cois,1,54,0,16,1,0
People's Party,8,113,4,231,0,124


In [38]:
feat_importances = pd.Series(rf_model.feature_importances_, index=X.columns)
feat_importances.sort_values(ascending=False, inplace=True)

In [39]:
pd.set_option("display.max_rows", None)
print(feat_importances)

cps19_lr_scale                                                                           0.037719
cps19_fed_gov_sat_Not at all satisfied                                                   0.035006
cps19_fed_gov_sat_Fairly satisfied                                                       0.034175
cps19_lib_promises_Strongly disagree                                                     0.028100
cps19_yob                                                                                0.027010
cps19_groups_therm_4                                                                     0.025547
cps19_income_number                                                                      0.025545
cps19_groups_therm_5                                                                     0.025338
cps19_snclav_Not at all well                                                             0.024866
cps19_groups_therm_3                                                                     0.024351
cps19_groups_therm_2

In [41]:
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.4677927188968036

In [14]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [15]:
y_pred = brf.predict(X_test)

In [16]:
balanced_accuracy_score(y_test, y_pred)

0.5434705186919203

In [17]:
feat_importances = pd.Series(brf.feature_importances_, index=X.columns)
feat_importances.sort_values(ascending=False,inplace=True)
pd.set_option("display.max_rows", None)
print(feat_importances)

cps19_yob                                                                                0.032011
cps19_lr_scale                                                                           0.030911
cps19_groups_therm_4                                                                     0.030750
cps19_income_number                                                                      0.029305
cps19_groups_therm_3                                                                     0.028812
cps19_groups_therm_5                                                                     0.028137
cps19_groups_therm_2                                                                     0.026719
cps19_groups_therm_1                                                                     0.026600
cps19_province_Quebec                                                                    0.024749
cps19_language_english_English                                                           0.019451
cps19_interest_elxn_